In [2]:
# ── Cell 1 ──
# Load environment and verify your REST API key

from dotenv import load_dotenv
import os

# Load kakao.env (must live in the same folder as this notebook)
load_dotenv('kakao.env')  

REST_API_KEY = os.getenv('KAKAO_REST_API_KEY')
if not REST_API_KEY:
    raise RuntimeError("❗ kakao.env에서 KAKAO_REST_API_KEY를 못 불러왔어요.")

print("✅ Kakao REST API Key loaded:", REST_API_KEY[:8] + "…")

✅ Kakao REST API Key loaded: d7d0c791…


In [4]:
import os, time, csv, requests
from dotenv import load_dotenv

# .env 로드
load_dotenv('kakao.env')
REST_API_KEY = os.getenv('KAKAO_REST_API_KEY')
if not REST_API_KEY:
    raise RuntimeError("❗ kakao.env에서 키를 로드할 수 없습니다.")

SEARCH_URL = 'https://dapi.kakao.com/v2/local/search/keyword.json'
headers    = {'Authorization': f'KakaoAK {REST_API_KEY}'}

# 서울 25개 구 목록
gus = [
  "강남구","강동구","강북구","강서구","관악구","광진구","구로구",
  "금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구",
  "서초구","성동구","성북구","송파구","양천구","영등포구","용산구",
  "은평구","종로구","중구","중랑구"
]

brands = ['KFC','맥도날드','맘스터치','롯데리아','버거킹']

def search_keyword(q, page):
    resp = requests.get(SEARCH_URL,
        headers=headers,
        params={'query':q, 'page':page, 'size':15})
    resp.raise_for_status()
    js = resp.json()
    return js['documents'], js['meta']['is_end']

all_places = {}
for brand in brands:
    for gu in gus:
        query = f"{brand} {gu}"
        page = 1
        while True:
            docs, is_end = search_keyword(query, page)
            for d in docs:
                # 중복 제거 키: id or place_name+address
                key = d['id'] if 'id' in d else d['place_name']+d['address_name']
                all_places.setdefault(key, {
                    'brand':       brand,
                    'place_name':  d['place_name'],
                    'address_name':d['address_name'],
                    'lng':          d['x'],
                    'lat':          d['y']
                })
            print(f"▶ {brand}·{gu} page{page}: {len(docs)}건")
            if is_end: break
            page += 1
            time.sleep(0.2)

# CSV 쓰기
with open('kakao_burger_all_seoul.csv','w',encoding='utf-8-sig',newline='') as f:
    w = csv.DictWriter(f,
        fieldnames=['brand','place_name','address_name','lng','lat'])
    w.writeheader()
    w.writerows(all_places.values())

print(f"✅ 완료! 총 {len(all_places)}건 저장됨")


▶ KFC·강남구 page1: 15건
▶ KFC·강남구 page2: 12건
▶ KFC·강동구 page1: 3건
▶ KFC·강북구 page1: 15건
▶ KFC·강서구 page1: 15건
▶ KFC·강서구 page2: 1건
▶ KFC·관악구 page1: 15건
▶ KFC·관악구 page2: 13건
▶ KFC·광진구 page1: 15건
▶ KFC·광진구 page2: 15건
▶ KFC·광진구 page3: 4건
▶ KFC·구로구 page1: 9건
▶ KFC·금천구 page1: 5건
▶ KFC·노원구 page1: 6건
▶ KFC·도봉구 page1: 12건
▶ KFC·동대문구 page1: 15건
▶ KFC·동대문구 page2: 14건
▶ KFC·동작구 page1: 13건
▶ KFC·마포구 page1: 15건
▶ KFC·마포구 page2: 9건
▶ KFC·서대문구 page1: 15건
▶ KFC·서대문구 page2: 7건
▶ KFC·서초구 page1: 15건
▶ KFC·서초구 page2: 15건
▶ KFC·서초구 page3: 1건
▶ KFC·성동구 page1: 15건
▶ KFC·성동구 page2: 15건
▶ KFC·성동구 page3: 7건
▶ KFC·성북구 page1: 15건
▶ KFC·송파구 page1: 15건
▶ KFC·송파구 page2: 13건
▶ KFC·양천구 page1: 15건
▶ KFC·양천구 page2: 9건
▶ KFC·영등포구 page1: 15건
▶ KFC·영등포구 page2: 13건
▶ KFC·용산구 page1: 3건
▶ KFC·은평구 page1: 15건
▶ KFC·은평구 page2: 1건
▶ KFC·종로구 page1: 8건
▶ KFC·중구 page1: 15건
▶ KFC·중구 page2: 15건
▶ KFC·중구 page3: 3건
▶ KFC·중랑구 page1: 15건
▶ KFC·중랑구 page2: 6건
▶ 맥도날드·강남구 page1: 15건
▶ 맥도날드·강남구 page2: 9건
▶ 맥도날드·강동구 page1: 5건
▶ 맥도날드·강북구 page1: 15건
▶ 맥